In [1]:
import os

import pprint
import numpy as np
import tensorflow as tf

from data import read_data, pad_data, depad_data
from model import MemN2N

pp = pprint.PrettyPrinter()

flags = tf.app.flags

flags.DEFINE_integer("edim", 20, "internal state dimension [20]")
flags.DEFINE_integer("nhop", 3, "number of hops [3]")
flags.DEFINE_integer("mem_size", 50, "maximum number of sentences that can be encoded into memory [50]")
flags.DEFINE_integer("batch_size", 32, "batch size to use during training [32]")
flags.DEFINE_integer("nepoch", 100, "number of epoch to use during training [100]")
flags.DEFINE_integer("anneal_epoch", 25, "anneal the learning rate every <anneal_epoch> epochs [25]")
flags.DEFINE_integer("babi_task", 7, "index of bAbI task for the network to learn [1]")
flags.DEFINE_float("init_lr", 0.01, "initial learning rate [0.01]")
flags.DEFINE_float("anneal_rate", 0.5, "learning rate annealing rate [0.5]")
flags.DEFINE_float("init_mean", 0., "weight initialization mean [0.]")
flags.DEFINE_float("init_std", 0.1, "weight initialization std [0.1]")
flags.DEFINE_float("max_grad_norm", 40, "clip gradients to this norm [40]")
flags.DEFINE_string("data_dir", "./bAbI/en-valid", "dataset directory [./bAbI/en_valid]")
flags.DEFINE_string("checkpoint_dir", "./checkpoints", "checkpoint directory [./checkpoints]")
flags.DEFINE_boolean("lin_start", False, "True for linear start training, False for otherwise [False]")
flags.DEFINE_boolean("is_test", False, "True for testing, False for training [False]")
flags.DEFINE_boolean("show_progress", False, "print progress [False]")

FLAGS = flags.FLAGS

word2idx = {}
max_words = 0
max_sentences = 0

if not os.path.exists(FLAGS.checkpoint_dir):
    os.makedirs(FLAGS.checkpoint_dir)

train_stories, train_questions, max_words, max_sentences = read_data('{}/qa{}_test.txt'.format(FLAGS.data_dir, FLAGS.babi_task), word2idx, max_words, max_sentences)
valid_stories, valid_questions, max_words, max_sentences = read_data('{}/qa{}_valid.txt'.format(FLAGS.data_dir, FLAGS.babi_task), word2idx, max_words, max_sentences)
test_stories, test_questions, max_words, max_sentences = read_data('{}/qa{}_test.txt'.format(FLAGS.data_dir, FLAGS.babi_task), word2idx, max_words, max_sentences)

pad_data(train_stories, train_questions, max_words, max_sentences)
pad_data(valid_stories, valid_questions, max_words, max_sentences)
pad_data(test_stories, test_questions, max_words, max_sentences)

idx2word = dict(zip(word2idx.values(), word2idx.keys()))
FLAGS.nwords = len(word2idx)
FLAGS.max_words = max_words
FLAGS.max_sentences = max_sentences

pp.pprint(flags.FLAGS.__flags)

with tf.Session() as sess:
    model = MemN2N(FLAGS, sess)
    model.build_model()

    if FLAGS.is_test:
        model.run(valid_stories, valid_questions, test_stories, test_questions)
    else:
        model.run(train_stories, train_questions, valid_stories, valid_questions)
    
    predictions, target = model.predict(train_stories, train_questions)

{'anneal_epoch': 25,
 'anneal_rate': 0.5,
 'babi_task': 7,
 'batch_size': 32,
 'checkpoint_dir': './checkpoints',
 'data_dir': './bAbI/en-valid',
 'edim': 20,
 'init_lr': 0.01,
 'init_mean': 0.0,
 'init_std': 0.1,
 'is_test': False,
 'lin_start': False,
 'max_grad_norm': 40,
 'max_sentences': 33,
 'max_words': 16,
 'mem_size': 50,
 'nepoch': 100,
 'nhop': 3,
 'nwords': 44,
 'show_progress': False}
{'learning_rate': 0.01, 'epoch': 0, 'loss': 1.3494583702087402, 'validation_loss': 1.170234591960907}
{'learning_rate': 0.01, 'epoch': 1, 'loss': 0.95766279029846191, 'validation_loss': 1.2053432273864746}
{'learning_rate': 0.01, 'epoch': 2, 'loss': 0.91837644863128665, 'validation_loss': 1.1492957496643066}
{'learning_rate': 0.01, 'epoch': 3, 'loss': 0.88957553339004514, 'validation_loss': 1.0938487553596496}
{'learning_rate': 0.01, 'epoch': 4, 'loss': 0.85897242641448979, 'validation_loss': 1.0075859284400941}
{'learning_rate': 0.01, 'epoch': 5, 'loss': 0.8295915842056274, 'validation_loss'

{'learning_rate': 0.0025, 'epoch': 74, 'loss': 0.45119453442096707, 'validation_loss': 0.59549145340919496}
{'learning_rate': 0.0025, 'epoch': 75, 'loss': 0.45521457958221434, 'validation_loss': 0.59324149847030638}
{'learning_rate': 0.00125, 'epoch': 76, 'loss': 0.4412108964920044, 'validation_loss': 0.57320029735565181}
{'learning_rate': 0.00125, 'epoch': 77, 'loss': 0.43739699697494505, 'validation_loss': 0.57419827461242678}
{'learning_rate': 0.00125, 'epoch': 78, 'loss': 0.43764109563827513, 'validation_loss': 0.57321208953857417}
{'learning_rate': 0.00125, 'epoch': 79, 'loss': 0.4348056709766388, 'validation_loss': 0.57525240898132324}
{'learning_rate': 0.00125, 'epoch': 80, 'loss': 0.43579065454006194, 'validation_loss': 0.57327941894531254}
{'learning_rate': 0.00125, 'epoch': 81, 'loss': 0.43294333553314207, 'validation_loss': 0.57508021354675298}
{'learning_rate': 0.00125, 'epoch': 82, 'loss': 0.43427425003051756, 'validation_loss': 0.57270117521286013}
{'learning_rate': 0.001

In [3]:
index = 25

depad_data(train_stories, train_questions)

question = train_questions[index]['question']
answer = train_questions[index]['answer']
story_index = train_questions[index]['story_index']
sentence_index = train_questions[index]['sentence_index']

story = train_stories[story_index][:sentence_index + 1]

story = [list(map(idx2word.get, sentence)) for sentence in story]
question = list(map(idx2word.get, question))
prediction = [idx2word[np.argmax(predictions[index])]]
answer = list(map(idx2word.get, answer))

print('Story:')
pp.pprint(story)
print('\nQuestion:')
pp.pprint(question)
print('\nPrediction:')
pp.pprint(prediction)
print('\nAnswer:')
pp.pprint(answer)
print('\nCorrect:')
pp.pprint(prediction == answer)


Story:
[['daniel', 'went', 'to', 'the', 'bathroom'],
 ['john', 'moved', 'to', 'the', 'office'],
 ['daniel', 'went', 'to', 'the', 'kitchen'],
 ['john', 'got', 'the', 'apple', 'there']]

Question:
['how', 'many', 'objects', 'is', 'john', 'carrying']

Prediction:
['one']

Answer:
['one']

Correct:
True
